# Lab2 - Data Collection and Pre-processing

## Done by Eris Leksi (9067882)

Hello! I am Eris and this is my Lab2 - data Collection and Pre-processing jupyter notebook. What are we going to do now in here? Nice question. Now we are going to dive into the data-collection and the pre-processing steps of machine learning. After chosing 2 datasets from the e-commerce and us government sectors, we have to load, clean and summarize them. Let's start!

Dataset 1: Sample - Superstore Sales

Type 1: XLS

Link 1: https://community.tableau.com/s/question/0D54T00000CWeX8SAL/sample-superstore-sales-excelxls

Description 1: A list of 9994 Superstole Sales accross the United States of America. It has 21 features including order_id, date, customer_id and also the type of purcased product and many more.

Dataset 2: United States Cities Database

Type 2: CSV

Link 2: https://simplemaps.com/data/us-cities

Description 2 (from the source): 

Up-to-date: Data updated as of January 23, 2025.

Comprehensive: Over 109,000 cities and towns from all 50 states, DC, Puerto Rico and the US Virgin Islands.

Useful fields: From latitude and longitude to household income.

Accurate: Aggregated and cleaned from official sources.



## 0. Importing the libraries

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from datetime import datetime
from collections import defaultdict
import json
import pyarrow.parquet as pq
import pyarrow as pa
import tabulate as tt

## 1. Hello Data!

In [ ]:
df_cities = pd.read_csv('uscities.csv')
                                          #loading the data from csv and excel file
df_sales = pd.read_excel('sales.xls')


# printing the first 3 rows and some extra information(of my choice) for each dataframe 
print("First 3 rows of df_cities:")
print(df_cities.head(3))
print("*----------*----------*----------*----------*-----------*-----------*")
print("df_cities table shape:")
print (df_cities.shape)
print("*----------*----------*----------*----------*-----------*-----------*")
print("df_cities columns description:")
print(df_cities.describe()) 
print("*----------*----------*----------*----------*-----------*-----------*")
print("df_cities rows description:")
print(df_cities.info())
print("First 3 rows of df_sales:")
print(df_sales.head(3))
print("*----------*----------*----------*----------*-----------*-----------*")
print("df_sales table shape:")
print(df_sales.shape)
print("*----------*----------*----------*----------*-----------*-----------*")
print("df_sales columns description:")
print(df_sales.describe())  
print("*----------*----------*----------*----------*-----------*-----------*")
print("df_sales rows description:")
print(df_sales.info())


First 3 rows of df_cities:
          city   city_ascii state_id  state_name  county_fips  county_name  \
0     New York     New York       NY    New York        36081       Queens   
1  Los Angeles  Los Angeles       CA  California         6037  Los Angeles   
2      Chicago      Chicago       IL    Illinois        17031         Cook   

       lat       lng  population  density source  military  incorporated  \
0  40.6943  -73.9249    18832416  10943.7  shape     False          True   
1  34.1141 -118.4068    11885717   3165.8  shape     False          True   
2  41.8375  -87.6866     8489066   4590.3  shape     False          True   

              timezone  ranking  \
0     America/New_York        1   
1  America/Los_Angeles        1   
2      America/Chicago        1   

                                                zips          id  
0  11229 11226 11225 11224 11223 11221 11220 1138...  1840034016  
1  90291 90293 90292 91316 91311 90035 90034 9003...  1840020491  
2  60018 6064

## 2. Pick the right container ##

In this lab I am going to use a ***class*** instead of a dictionary or a tuple. It offers encapsulation for both data and behaviours including cleaning, parsing, computing, which makes it even better for future expansion/implementation.

## Extra: Reduce Dimensionality

Since we have a lot of features which we do not actually need, a good standard is to drop some of the columns or as said in the machine learning language, reduce dimensionality.

In [11]:
df_cities = df_cities.drop(['state_name', 'city_ascii', 'military', 'incorporated', 'timezone', 'ranking', 'source'], axis=1)

df_sales = df_sales.drop(['Customer Name', 'Country', 'Product Name'], axis=1)

## 3. Transaction Class and OO data structure

In [ ]:
class Transaction:
    def __init__ (self, row):
        self.order_id = row["Order ID"]
        self.order_date = row["Order Date"]
        self.ship_date = row["Ship Date"]
        self.customer_id = row["Customer ID"]
        self.product = row["Product ID"]
        self.price = row["Sales"]
        self.quantity = row["Quantity"]
        self.discount = row["Discount"]
        self.city = row["City"]
        self.clean()

    def clean (self):
        # convert date
        if isinstance(self.order_date, str):
            self.order_date = pd.to_datetime(self.order_date)
        # make sure price and quantity type float and int
        self.price = float(self.price)
        self.quantity = int(self.quantity)
        self.discount = float(self.discount)

    def total(self):
        return round(self.price * self.quantity * (1 - self.discount), 2)
    
class Cities:
    def __init__(self, row):
        self.city = row["city"]
        self.state_id = row["state_id"]
        self.state_name = row["state_name"]
        self.population = row["population"]
        self.lat = row["lat"]
        self.lng = row["lng"]
        self.military = row["military"]
        self.incorporated = row["incorporated"]
        self.timezone = row["timezone"]
        self.ranking = row["ranking"]
        self.source = row["source"] 

## 4. Bulkloader

In [ ]:
def load(df) -> list[Transaction]:
    return [Transaction(row) for _, row in df.iterrows()]

transactions = load(df_sales)
print(f"Loaded {len(transactions)} transactions")

def load_cities(df) -> list[Cities]:
    return [Cities(row) for _, row in df.iterrows()]    

Loaded 9994 transactions


## 5. Quick Profiling

In [51]:
prices = [t.price for t in transactions]
cities = set(t.city for t in transactions)
print(f"Total Price: {sum(prices)}")
print(f"Minimal Price: {min(prices)}")
print(f"Mean Price: {sum(prices)/len(prices)}")
print(f"Max Price: {max(prices)}")
print(f"Unique shipping cities: {len(cities)}")

Total Price: 2297200.8603
Minimal Price: 0.44399999999999995
Mean Price: 229.85800083049827
Max Price: 22638.48
Unique shipping cities: 531


## 6. Spot the Grime

In [52]:
print("Checking for negative prices:", any(t.price < 0 for t in transactions))
print("Checking for missing city:", any(t.city == "" for t in transactions))
print("Checking for invalid quantity:", any(t.quantity <= 0 for t in transactions))

Checking for negative prices: False
Checking for missing city: False
Checking for invalid quantity: False


## 7. Cleaning Rules

In [53]:
beforeCount = len(transactions)
transactions = [t for t in transactions if t.price >= 0 and t.quantity > 0 and t.city != ""]
afterCount = len(transactions)
print(f"Before cleaning: {beforeCount}")
print(f"After cleaning: {afterCount}")

if beforeCount == afterCount:
    print("The data is already clean, no need to remove any rows.")

Before cleaning: 9994
After cleaning: 9994
The data is already clean, no need to remove any rows.


## 8. Transformations

In [54]:
for t in transactions:
    t.discount1 = t.discount * 100
print("Discount values have been converted to percentage.")

Discount values have been converted to percentage.


## 9. Feature Engineering

In [55]:
for t in transactions:
    if isinstance(t.order_date, str):
        t.order_date = pd.to_datetime(t.order_date)

    if isinstance(t.ship_date, str):
        t.ship_date = pd.to_datetime(t.ship_date)

print("Order and ship dates have been converted to datetime objects.")

Order and ship dates have been converted to datetime objects.


## 8.2. Transformation 2 
Done down here because we firstly had to convert the date into datetime format using the Timestamp library

In [60]:
for t in transactions:
    t.processingTime = (t.ship_date - t.order_date).days

print("Processing time has been calculated for each transaction.")
   

Processing time has been calculated for each transaction.


## 10. Mini-Aggregation

In [61]:
cityRevenue = defaultdict(float)
for t in transactions:
    cityRevenue[t.city] += t.total()

# display top 5 cities by revenue
sortedRevenue = sorted(cityRevenue.items(), key=lambda x: x[1], reverse=True)
print("Top 5 Cities by Revenue:")
for city, revenue in sortedRevenue[:5]:
    print(f"{city}: ${revenue}")

Top 5 Cities by Revenue:
New York City: $1167991.6800000002
Los Angeles: $765588.9899999994
Seattle: $557065.6400000004
San Francisco: $479247.78
Philadelphia: $380496.1100000001


## 11. Serialization Checkpoint

In [62]:
cleaned_data = [
    {
        "Order ID": t.order_id,
        "Product": t.product,
        "Unit Price": t.price,
        "Quantity": t.quantity,
        "Discount Code": t.discount,
        "Discount (%)": t.discount1,
        "Total Amount": t.total(),
        "Days Since Purchase": t.processingTime,
        "Shipping City": t.city
    }
    for t in transactions
]

with open("cleaned.json", "w") as f:
    json.dump(cleaned_data, f, indent=2)

print("Cleaned transaction data has been saved to 'cleaned.json'")

table = pa.Table.from_pandas(pd.DataFrame(cleaned_data))
pq.write_table(table, "cleaned1.parquet")

print("Cleaned transaction data has been saved to 'cleaned1.parquet'")


Cleaned transaction data has been saved to 'cleaned.json'
Cleaned transaction data has been saved to 'cleaned1.parquet'


## 12. Soft Interview Reflection

Using Object-Oriented Programming (OOP) was a game-changer for this project. It let me wrap up all the related actions and logic neatly inside classes, which meant I could easily reuse them whenever I needed to. This saved a lot of time and effort down the line.
Another big win for using classes was how much cleaner and more organized the code became. Instead of having to repeat the same steps over and over to clean, change, and improve each piece of data, the class structure kept everything tidy. It made handling all those transactions much smoother and kept the whole system easy to manage.

## Data Dictionary

In [64]:
data = [
    ["order_id", "String", "Unique identifier for each order", "sales.xls"],
    ["product", "String", "Name or category of the product purchased", "sales.xls"],
    ["price", "Float", "Selling price of a single product unit", "sales.xls"],
    ["quantity", "Integer", "Number of units purchased in the order", "sales.xls"],
    ["discount", "String", "Promo code or discount tag applied", "sales.xls"],
    ["discount1", "Float", "Discount percentage parsed from the promo code", "derived"],
    ["total", "Float", "Total amount after applying quantity and discount", "calculated"],
    ["dayCount", "Integer", "Number of days since the transaction took place", "derived"],
    ["city", "String", "Shipping city for the order", "sales.xls"],
    ["order_date", "DateTime", "Date when the order was placed", "sales.xls"],
    ["ship_date", "DateTime", "Date when the order was shipped", "sales.xls"],
    ["processingTime", "Integer", "Number of days taken to process the order", "derived"],
    ["state_id", "String", "Two-letter abbreviation of the U.S. state", "derived from uscities.csv"],
    ["lat", "Float", "Latitude coordinate of the shipping city", "derived from uscities.csv"],
    ["lng", "Float", "Longitude coordinate of the shipping city", "derived from uscities.csv"],
    ["population", "Integer", "Approximate population of the shipping city", "derived from uscities.csv"],
    ["state_id", "String", "Two-letter abbreviation of the U.S. state", "uscities.csv"],
    ["lat", "Float", "Latitude coordinate of the shipping city", "uscities.csv"],
    ["lng", "Float", "Longitude coordinate of the shipping city", "uscities.csv"],
    ["population", "Integer", "Approximate population of the shipping city", "uscities.csv"],
]


df_table = pd.DataFrame(data, columns=["Field", "Type", "Description", "Source"])

print(tt.tabulate(df_table, headers='keys', tablefmt='grid'))


+----+----------------+----------+---------------------------------------------------+---------------------------+
|    | Field          | Type     | Description                                       | Source                    |
+====+================+==========+===================================================+===========================+
|  0 | order_id       | String   | Unique identifier for each order                  | sales.xls                 |
+----+----------------+----------+---------------------------------------------------+---------------------------+
|  1 | product        | String   | Name or category of the product purchased         | sales.xls                 |
+----+----------------+----------+---------------------------------------------------+---------------------------+
|  2 | price          | Float    | Selling price of a single product unit            | sales.xls                 |
+----+----------------+----------+----------------------------------------------

Here is the version of it as a Markdown: 

___________________________________________________________________________________________________________________
| #   | Field          | Type     | Description                                       | Source                    |
|-----|----------------|----------|---------------------------------------------------|---------------------------|
| 0   | order_id       | String   | Unique identifier for each order                  | sales.xls                 |
| 1   | product        | String   | Name or category of the product purchased         | sales.xls                 |
| 2   | price          | Float    | Selling price of a single product unit            | sales.xls                 |
| 3   | quantity       | Integer  | Number of units purchased in the order            | sales.xls                 |
| 4   | discount       | String   | Promo code or discount tag applied                | sales.xls                 |
| 5   | discount1      | Float    | Discount percentage parsed from the promo code    | derived                   |
| 6   | total          | Float    | Total amount after applying quantity and discount | calculated                |
| 7   | dayCount       | Integer  | Number of days since the transaction took place   | derived                   |
| 8   | city           | String   | Shipping city for the order                       | sales.xls                 |
| 9   | order_date     | DateTime | Date when the order was placed                    | sales.xls                 |
| 10  | ship_date      | DateTime | Date when the order was shipped                   | sales.xls                 |
| 11  | processingTime | Integer  | Number of days taken to process the order         | derived                   |
| 12  | state_id       | String   | Two-letter abbreviation of the U.S. state         | derived from uscities.csv |
| 13  | lat            | Float    | Latitude coordinate of the shipping city          | derived from uscities.csv |
| 14  | lng            | Float    | Longitude coordinate of the shipping city         | derived from uscities.csv |
| 15  | population     | Integer  | Approximate population of the shipping city       | derived from uscities.csv |
| 16  | state_id       | String   | Two-letter abbreviation of the U.S. state         | uscities.csv              |
| 17  | lat            | Float    | Latitude coordinate of the shipping city          | uscities.csv              |
| 18  | lng            | Float    | Longitude coordinate of the shipping city         | uscities.csv              |
| 19  | population     | Integer  | Approximate population of the shipping city       | uscities.csv              |
-------------------------------------------------------------------------------------------------------------------
